In [29]:
# importing various libararies and downloading dependencies

import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes

import folium
print ("Libraries imported")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported


In [3]:
# reading previously saved csv file to dataframe
df_Toronto = pd.read_csv("Toronto3.csv", index_col=0)
df_Toronto

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Port Union,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Highland Creek,43.784535,-79.160497
5,M1E,Scarborough,Morningside,43.763573,-79.188711
6,M1E,Scarborough,West Hill,43.763573,-79.188711
7,M1E,Scarborough,Guildwood,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
#Focusing analysis on Downtown Toronto
DowntownT_data = df_Toronto[df_Toronto['Borough'] == "Downtown Toronto"].reset_index(drop=True)
DowntownT_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
3,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [17]:
#Obtaining geographical coordinates of Downtown Toronto
address = "Downtown Toronto, Canada"

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


/anaconda3/lib/python3.7/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [21]:
#creating map of Toronto and adding markers of neighborhoods to map
map_toronto=folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(DowntownT_data['Latitude'], DowntownT_data['Longitude'],
                           DowntownT_data['Neighborhood']):
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [22]:
#Define Foursquare credentials and version
CLIENT_ID = 'AVTAIQFV3C22ZFIZGUBZE3IOU0J2HLMVU1QT32QNKCXAMRCW'
CLIENT_SECRET = '3EEBKPNKLPZPHCWBCRLCPE5NX1HHOGNEHPQMTXKBCQX44SDJ'
VERSION = '20180605'

In [23]:
#Exploring first neighborhood

DowntownT_data.loc[0,'Neighborhood']

'Harbourfront'

In [24]:
neighborhood_latitude = DowntownT_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DowntownT_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DowntownT_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [25]:
#Top 100 venues in Harbourfront
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=AVTAIQFV3C22ZFIZGUBZE3IOU0J2HLMVU1QT32QNKCXAMRCW&client_secret=3EEBKPNKLPZPHCWBCRLCPE5NX1HHOGNEHPQMTXKBCQX44SDJ&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [26]:
#results of URL
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0809ee1ed2194a7734de88'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [30]:
#clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [32]:
#Create function to repeat the same process to all neighborhoods in Downtown Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

DowntownT_venues = getNearbyVenues(names=DowntownT_data['Neighborhood'],
                                   latitudes=DowntownT_data['Latitude'],
                                   longitudes=DowntownT_data['Longitude']
                                  )



Harbourfront
Regent Park
Garden District
Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide
King
Richmond
Toronto Islands
Union Station
Harbourfront East
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Kensington Market
Grange Park
CN Tower
South Niagara
Railway Lands
King and Spadina
Harbourfront West
Island airport
Bathurst Quay
Christie


In [33]:
#get shape of dataframe 
print(DowntownT_venues.shape)
DowntownT_venues.head()


(2018, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [34]:
# number of venues returned for each neightborhood

DowntownT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,55,55,55,55,55,55
CN Tower,14,14,14,14,14,14
Central Bay Street,84,84,84,84,84,84
Chinatown,100,100,100,100,100,100
Christie,15,15,15,15,15,15
Commerce Court,100,100,100,100,100,100
Design Exchange,100,100,100,100,100,100


In [35]:
#unique categories
print('There are {} uniques categories.'.format(len(DowntownT_venues['Venue Category'].unique())))

There are 182 uniques categories.


In [41]:
#analyzing each neighborhood
# one hot encoding
DowntownT_onehot = pd.get_dummies(DowntownT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DowntownT_onehot['Neighborhood'] = DowntownT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DowntownT_onehot.columns[-1]] + list(DowntownT_onehot.columns[:-1])
DowntownT_onehot = DowntownT_onehot[fixed_columns]

DowntownT_onehot.head()


,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
DowntownT_onehot.shape

(2018, 182)

In [43]:
#group by neighborhood by taking the mean of the frequency of occurance of each category

DowntownT_grouped = DowntownT_onehot.groupby('Neighborhood').mean().reset_index()
DowntownT_grouped


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.040000,0.010000,0.00,0.00,0.010000,0.000000,0.00,0.010000,0.01
1,Bathurst Quay,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.018182,0.018182,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00
3,CN Tower,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00
4,Central Bay Street,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,...,0.011905,0.023810,0.000000,0.00,0.00,0.011905,0.000000,0.00,0.011905,0.00
5,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.00,0.00,0.050000,0.000000,0.05,0.010000,0.00
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00
7,Commerce Court,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.010000,0.00
8,Design Exchange,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.010000,0.020000,0.010000,0.00,0.01,0.000000,0.000000,0.00,0.010000,0.00
9,Garden District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.020000,0.020000,0.020000,0.01,0.00,0.010000,0.000000,0.01,0.010000,0.00


In [46]:
#print top 5 venues for each neighborhood

num_top_venues = 5

for hood in DowntownT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DowntownT_grouped[DowntownT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Bathurst Quay----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3     Harbor / Marina  0.07
4  Airport Food Court  0.07


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2      Restaurant  0.05
3             Pub  0.04
4  Farmers Market  0.04


----CN Tower----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3     Harbor / Marina  0.07
4  Airport Food Court  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.06
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.04


----Chinatown----
                           venue  freq
0       

In [47]:
#sort venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DowntownT_grouped['Neighborhood']

for ind in np.arange(DowntownT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DowntownT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Gym,Burger Joint,Restaurant,Hotel,Bar
1,Bathurst Quay,Airport Terminal,Airport Lounge,Airport Service,Sculpture Garden,Plane,Boat or Ferry,Boutique,Harbor / Marina,Airport Gate,Airport Food Court
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Cheese Shop,Café,Pub
3,CN Tower,Airport Terminal,Airport Lounge,Airport Service,Sculpture Garden,Plane,Boat or Ferry,Boutique,Harbor / Marina,Airport Gate,Airport Food Court
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Spa,Japanese Restaurant
5,Chinatown,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Dessert Shop
6,Christie,Grocery Store,Café,Park,Diner,Italian Restaurant,Baby Store,Restaurant,Convenience Store,Coffee Shop,Nightclub
7,Commerce Court,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gastropub,Seafood Restaurant,Steakhouse,Gym,Deli / Bodega
8,Design Exchange,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Steakhouse,Deli / Bodega,Italian Restaurant,Gym,Gastropub
9,Garden District,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Burger Joint,Ramen Restaurant


In [49]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

DowntownT_grouped_clustering = DowntownT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DowntownT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 1, 0, 1, 0, 2, 4, 0, 0, 0], dtype=int32)

In [50]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
DowntownT_merged = DowntownT_data

# add clustering labels
DowntownT_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DowntownT_merged = DowntownT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DowntownT_merged.head() 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Café,Theater,Breakfast Spot,Gym / Fitness Center,Electronics Store
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Café,Theater,Breakfast Spot,Gym / Fitness Center,Electronics Store
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Burger Joint,Ramen Restaurant
3,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Burger Joint,Ramen Restaurant
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Cocktail Bar,Clothing Store,Cosmetics Shop,Bakery,Gastropub


In [52]:
#Visualize clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DowntownT_merged['Latitude'], DowntownT_merged['Longitude'], DowntownT_merged['Neighborhood'], DowntownT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
